In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые,
# и класс GridSearchCV, позволяющий выполнить 
# поиск по сетке
from sklearn.model_selection import (train_test_split,
                                     GridSearchCV)
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс Pipeline, позволяющий 
# создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем класс PowerTransformer, позволяющий 
# выполнить преобразование Бокса-Кокса/Йео-Джонсона
# и стандартизацию, класс OneHotEncoder, позволяющий 
# выполнить дамми-кодирование
from sklearn.preprocessing import (PowerTransformer, 
                                   OneHotEncoder)
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression
# импортируем функцию roc_auc_score()
# для вычисления AUC-ROC
from sklearn.metrics import roc_auc_score
# импортируем собственный класс Replacer
from utils import Replacer
# увеличиваем количество отображаемых строк
pd.set_option('display.max_rows', 200)

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Verizon.csv', sep=';')
# смотрим первые 5 наблюдений
data.head()

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,8.62,NaN,8.49,Нет,Бюджетный,CH,43.0,Мужской,_Женат,0.0,33935.8,0
1,21.27,0.0,"218,12",Нет,Бюджетный,CH,60.0,NaN,_Одинокий,2.0,"95930,6",1
2,"6,13",0.0,NaN,Да,NaN,NaN,25.0,Женский,NaN,2.0,"295,34",1
3,16.46,0.0,"57,66",Да,Бесплатный,NaN,93.0,Женский,Одинокий,0.0,NaN,1
4,NaN,0.0,"16,01",Да,Бесплатный,CC,68.0,Женский&*,NaN,0.0,"99832,9",1


In [3]:
# посмотрим наличие дублей
data[data.duplicated(keep=False)]

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,8.62,NaN,8.49,Нет,Бюджетный,CH,43.0,Мужской,_Женат,0.0,33935.8,0
13,8.62,NaN,8.49,Нет,Бюджетный,CH,43.0,Мужской,_Женат,0.0,33935.8,0
14,8.62,NaN,8.49,Нет,Бюджетный,CH,43.0,Мужской,_Женат,0.0,33935.8,0


In [4]:
# удаляем дубли на месте, оставляя первое
# встретившееся наблюдение в паттерне дубля
data.drop_duplicates(subset=None, keep='first', 
                     inplace=True)

In [5]:
# создаем список переменных
cols_lst = data.columns.tolist()
# записываем количество уникальных 
# значений по каждой переменной
uniq = [data[col].nunique() for col in cols_lst]
# записываем тип каждой переменной
types = data.dtypes
pd.DataFrame({'type': types, 'n_uniq': uniq})

,type,n_uniq
longdist,object,1081
internat,object,218
local,object,1372
int_disc,object,2
billtype,object,2
pay,object,4
age,float64,80
gender,object,4
marital,object,5
children,float64,3


In [6]:
# заменяем запятые на точки и преобразуем в тип float
for col in ['longdist', 'internat', 'local', 'income']:
    data[col] = data[col].str.replace(',', '.').astype('float')

In [7]:
# посмотрим типы переменных и количество 
# непропущенных значений
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1476
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   longdist  1467 non-null   float64
 1   internat  1469 non-null   float64
 2   local     1466 non-null   float64
 3   int_disc  1471 non-null   object 
 4   billtype  1450 non-null   object 
 5   pay       1470 non-null   object 
 6   age       1473 non-null   float64
 7   gender    1469 non-null   object 
 8   marital   1471 non-null   object 
 9   children  1474 non-null   float64
 10  income    1471 non-null   float64
 11  churn     1475 non-null   int64  
dtypes: float64(6), int64(1), object(5)
memory usage: 149.8+ KB


In [8]:
# смотрим пропуски в переменных
data.isnull().sum()

longdist     8
internat     6
local        9
int_disc     4
billtype    25
pay          5
age          2
gender       6
marital      4
children     1
income       4
churn        0
dtype: int64

In [9]:
# создаем список категориальных переменных
cat_cols = data.select_dtypes(
    include=['object']).columns.tolist()
# смотрим уникальные значения категориальных переменных
for col in cat_cols:
    print(col, data[col].unique())

int_disc ['Нет' 'Да' nan]
billtype ['Бюджетный' nan 'Бесплатный']
pay ['CH' nan 'CC' 'CD' 'Auto']
gender ['Мужской' nan 'Женский' 'Женский&*' 'Мужской&*']
marital ['_Женат' '_Одинокий' nan 'Одинокий' 'Женат' 'Же&нат']


In [10]:
# удаляем лишние символы в категориях 
# переменных gender и marital
for col in ['gender', 'marital']:
    data[col] = data[col].str.replace('[*&_]', '', regex=True)
    
# проверяем
for col in ['gender', 'marital']:
    print(col, data[col].unique())

gender ['Мужской' nan 'Женский']
marital ['Женат' 'Одинокий' nan]


In [11]:
# создаем список категориальных переменных
cat_cols = data.select_dtypes(
    include=['object']).columns.tolist()

# смотрим частоты по категориальным переменным, 
# чтобы выявить редкие категории
for col in cat_cols:
    print(data[col].value_counts(dropna=False))
    print('')

Нет    1015
Да      456
NaN       4
Name: int_disc, dtype: int64

Бюджетный     731
Бесплатный    719
NaN            25
Name: billtype, dtype: int64

CC      846
CH      324
Auto    297
NaN       5
CD        3
Name: pay, dtype: int64

Женский    743
Мужской    726
NaN          6
Name: gender, dtype: int64

Женат       872
Одинокий    599
NaN           4
Name: marital, dtype: int64



In [12]:
# заменяем редкую категорию модой
data.loc[data['pay'] == 'CD', 'pay'] = 'CC'

In [13]:
# создаем переменную - взаимодействие
data['gender_marital'] = np.where(
    (data['gender'].isnull()) | (data['marital'].isnull()), 
    np.NaN,  
    data.apply(lambda x: f"{x['gender']} + {x['marital']}", axis=1))

In [14]:
# поделим возраст на длительность междугородних звонков в минутах
cond = (data['age'] == 0) | (data['longdist'] == 0)
data['ratio'] = np.where(cond, 0, data['age'] / data['longdist'])

In [15]:
# поделим длительность междугородних звонков в минутах на
# длительность международных звонков в минутах
cond = (data['longdist'] == 0) | (data['internat'] == 0)
data['ratio2'] = np.where(cond, 0, data['longdist'] / data['internat'])

In [16]:
# поделим доход на возраст
cond = (data['income'] == 0) | (data['age'] == 0)
data['ratio3'] = np.where(cond, 0, data['income'] / data['age'])

In [17]:
# поделим возраст на количество детей
cond = (data['age'] == 0) | (data['children'] == 0)
data['ratio4'] = np.where(cond, 0, data['age'] / data['children'])

In [18]:
# разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('churn', axis=1), 
    data['churn'],
    test_size=0.3,
    stratify=data['churn'],
    random_state=30)

In [19]:
# создаем список категориальных признаков
# и список количественных признаков
cat_columns = X_train.select_dtypes(
    include='object').columns.tolist()
num_columns = X_train.select_dtypes(
    exclude='object').columns.tolist()

In [20]:
# создаем трансформер для категориальных переменных
cat_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, 
                          handle_unknown='ignore'))
])

In [21]:
# создаем трансформер для количественных переменных
num_pipe = Pipeline([
    ('imp', SimpleImputer()),
    ('repl', Replacer(repl_value=0.1)),
    ('boxcox', PowerTransformer(method='box-cox', 
                                standardize=True))
])

In [22]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название трансформера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

# передаем список в ColumnTransformer
ct = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
ml_pipe = Pipeline([
    ('tr', ct), 
    ('logreg', LogisticRegression(solver='liblinear'))
])

In [23]:
# обучаем конвейер базовых моделей
ml_pipe.fit(X_train, y_train)
# оцениваем качество конвейера на обучающих данных
print("AUC-ROC на обучающей выборке: {:.3f}".format(
    roc_auc_score(y_train, ml_pipe.predict_proba(X_train)[:, 1])))
# оцениваем качество конвейера на тестовых данных
print("AUC-ROC на тестовой выборке: {:.3f}".format(
    roc_auc_score(y_test, ml_pipe.predict_proba(X_test)[:, 1])))

AUC-ROC на обучающей выборке: 0.885
AUC-ROC на тестовой выборке: 0.861


In [24]:
# задаем сетку значений гиперпараметров
param_grid = {
    'tr__num__imp__strategy': ['mean', 'median', 'constant'],
    'tr__num__repl__repl_value': [0.1, 0.2, 0.3],
    'tr__cat__imp__strategy': ['most_frequent', 'constant'],
    'logreg__C': np.logspace(-2, 1, 10)
}
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров, количество блоков
# перекрестной проверки, метрику
gs = GridSearchCV(ml_pipe, 
                  param_grid, 
                  cv=5, 
                  scoring='roc_auc')
# выполняем поиск по сетке
gs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs.best_params_))
# смотрим наилучшее значение AUC-ROC
print("Наилучшее значение AUC-ROC: {:.3f}".format(
    gs.best_score_))
# смотрим AUC-ROC на тестовой выборке
print("Значение AUC-ROC на тестовой выборке: {:.3f}".format(
    roc_auc_score(y_test, gs.predict_proba(X_test)[:, 1])))

Наилучшие значения гиперпараметров:
{'logreg__C': 0.46415888336127775, 'tr__cat__imp__strategy': 'constant', 'tr__num__imp__strategy': 'constant', 'tr__num__repl__repl_value': 0.3}
Наилучшее значение AUC-ROC: 0.879
Значение AUC-ROC на тестовой выборке: 0.866


In [25]:
# запишем результаты поиска в DataFrame
results = pd.DataFrame(gs.cv_results_)
# превращаем в сводную таблицу
table = results.pivot_table(
    values=['mean_test_score'],    
    index=['param_logreg__C',
           'param_tr__num__imp__strategy',
           'param_tr__num__repl__repl_value',
           'param_tr__cat__imp__strategy'])
table.sort_values('mean_test_score', 
                  ascending=False, 
                  inplace=True)
table

,,,,mean_test_score
param_logreg__C,param_tr__num__imp__strategy,param_tr__num__repl__repl_value,param_tr__cat__imp__strategy,
0.464159,constant,0.3,constant,0.879293
1.000000,constant,0.3,constant,0.879219
10.000000,constant,0.3,most_frequent,0.879077
4.641589,constant,0.3,most_frequent,0.879038
2.154435,constant,0.3,most_frequent,0.878941
1.000000,constant,0.3,most_frequent,0.878845
0.464159,constant,0.3,most_frequent,0.878823
0.215443,constant,0.3,constant,0.878776
1.000000,median,0.3,constant,0.878671


In [26]:
# записываем CSV-файл в объект DataFrame 
fulldata = pd.read_csv('Data/Verizon.csv', sep=';')

In [27]:
# пишем функцию, выполняющую предварительную 
# обработку всех исторических данных
def preprocessing(df):
    # удаляем дубли на месте, оставляя первое
    # встретившееся наблюдение в паттерне дубля
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    # заменяем запятые на точки и преобразуем в тип float
    for i in ['longdist', 'internat', 'local', 'income']:
        df[i] = df[i].str.replace(',', '.').astype('float')
    # удаляем возможные лишние символы и цифры в категориях 
    # переменных gender и marital
    for i in ['gender', 'marital']:
        df[i] = df[i].str.replace('[\d+\W_]', '', regex=True)
    # все новые категории переменной pay заменяем модой
    lst = ['CC', 'Auto', 'CH', np.NaN]
    replace_new_values = lambda x: 'CC' if x not in lst else x
    df['pay'] = df['pay'].map(replace_new_values)
    # создаем переменную - результат конъюнкции  
    df['gender_marital'] = np.where(
        (df['gender'].isnull()) | (df['marital'].isnull()), 
        np.NaN,  
        df.apply(lambda x: f"{x['gender']} + {x['marital']}", axis=1))   
    # поделим возраст на длительность междугородних звонков в минутах
    cond = (df['age'] == 0) | (df['longdist'] == 0)
    df['ratio'] = np.where(cond, 0, df['age'] / df['longdist'])
    # поделим длительность междугородних звонков в минутах на
    # длительность международных звонков в минутах
    cond = (df['longdist'] == 0) | (df['internat'] == 0)
    df['ratio2'] = np.where(cond, 0, df['longdist'] / df['internat'])
    # поделим доход на возраст
    cond = (df['income'] == 0) | (df['age'] == 0)
    df['ratio3'] = np.where(cond, 0, df['income'] / df['age'])
    # поделим возраст на количество детей
    cond = (df['age'] == 0) | (df['children'] == 0)
    df['ratio4'] = np.where(cond, 0, df['age'] / df['children'])
    return df

In [28]:
# применяем функцию предварительной обработки 
# ко всем историческим данным
fulldata = preprocessing(fulldata)
fulldata.head()

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn,gender_marital,ratio,ratio2,ratio3,ratio4
0,8.62,NaN,8.49,Нет,Бюджетный,CH,43.0,Мужской,Женат,0.0,33935.80,0,Мужской + Женат,4.988399,NaN,789.204651,0.0
1,21.27,0.0,218.12,Нет,Бюджетный,CH,60.0,NaN,Одинокий,2.0,95930.60,1,NaN,2.820874,0.0,1598.843333,30.0
2,6.13,0.0,NaN,Да,NaN,NaN,25.0,Женский,NaN,2.0,295.34,1,NaN,4.078303,0.0,11.813600,12.5
3,16.46,0.0,57.66,Да,Бесплатный,NaN,93.0,Женский,Одинокий,0.0,NaN,1,Женский + Одинокий,5.650061,0.0,NaN,0.0
4,NaN,0.0,16.01,Да,Бесплатный,CC,68.0,Женский,NaN,0.0,99832.90,1,NaN,NaN,0.0,1468.130882,0.0


In [29]:
# создаем массив меток и массив признаков
y_fulldata = fulldata.pop('churn')

In [30]:
# записываем оптимальные значения гиперпараметров
best_params = gs.best_params_
# присваиваем итоговому конвейеру оптимальные 
# значения гиперпараметров
ml_pipe.set_params(**best_params)
# обучаем итоговый конвейер с оптимальными значениями 
# гиперпараметров на всех исторических данных
ml_pipe.fit(fulldata, y_fulldata);

In [31]:
# записываем CSV-файл, содержащий новые данные,
# в объект DataFrame
newdata = pd.read_csv('Data/Verizon_new.csv', sep=';')
newdata.head()

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income
0,"19,46","7,11","64,51",Нет,Бюджетный,CH,28.0,Женский7,Одинокий,1.0,43787
1,"18,42","6,46","28,07",Да,Бесплатный,CH,35.0,Женский&*,Женат,2.0,"47396,2"
2,"10,97",0,60,Да,Бесплатный,CH,45.0,Мужской$,Женат,0.0,"30057,4"
3,"22,18",0,"149,73",Нет,Бесплатный,HH,41.0,Мужской$,Одинокий,1.0,"21126,7"
4,0,0,"4,15",Да,Бесплатный,CH,40.0,Женский&*,_Одинокий,2.0,"81000,9"


In [32]:
# применяем функцию предварительной обработки 
# к новым данным
newdata = preprocessing(newdata)
newdata.head()

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,gender_marital,ratio,ratio2,ratio3,ratio4
0,19.46,7.11,64.51,Нет,Бюджетный,CH,28.0,Женский,Одинокий,1.0,43787.0,Женский + Одинокий,1.438849,2.736990,1563.821429,28.0
1,18.42,6.46,28.07,Да,Бесплатный,CH,35.0,Женский,Женат,2.0,47396.2,Женский + Женат,1.900109,2.851393,1354.177143,17.5
2,10.97,0.00,60.00,Да,Бесплатный,CH,45.0,Мужской,Женат,0.0,30057.4,Мужской + Женат,4.102097,0.000000,667.942222,0.0
3,22.18,0.00,149.73,Нет,Бесплатный,CC,41.0,Мужской,Одинокий,1.0,21126.7,Мужской + Одинокий,1.848512,0.000000,515.285366,41.0
4,0.00,0.00,4.15,Да,Бесплатный,CH,40.0,Женский,Одинокий,2.0,81000.9,Женский + Одинокий,0.000000,0.000000,2025.022500,20.0


In [33]:
# пишем функцию, проверяющую порядок переменных в наборах
def check_vars_order(hist_data, new_data):
    if hist_data == new_data:
        print("Одни и те же списки переменных, " + 
              "один и тот же порядок.")
    elif sorted(hist_data) == sorted(new_data):
        print("Одни и те же списки переменных, " + 
              "разный порядок.")
    else:
        print("Совершенно разные списки переменных.")  

In [34]:
# создаем списки переменных для исторического набора
# и набора новых данных
fulldata_cols = fulldata.columns.tolist()
newdata_cols = newdata.columns.tolist()

# выполняем проверку совпадения порядка столбцов
# в историческом наборе и наборе новых данных
check_vars_order(fulldata_cols, newdata_cols) 

Одни и те же списки переменных, один и тот же порядок.


In [35]:
# при помощью итогового конвейера с оптимальными значениями 
# гиперпараметров, обученного на всей исторической выборке, 
# вычисляем вероятности классов для новых данных
proba = ml_pipe.predict_proba(newdata)
# выведем вероятности классов для первых 5 наблюдений
proba[:5]

array([[0.04534913, 0.95465087],
       [0.05034639, 0.94965361],
       [0.9233456 , 0.0766544 ],
       [0.9238732 , 0.0761268 ],
       [0.03992666, 0.96007334]])